In [1]:
import pandas as pd
import numpy as np
from sklearn import metrics
from sklearn.model_selection import train_test_split
from sklearn.metrics import r2_score
from sklearn.ensemble import RandomForestRegressor

In [56]:
def Model_trainer_RandomForest(df, dummies_columns):
    def prep_data(df, nan_columns):    
        df = df.dropna(subset=nan_columns) #['stm_hers_tijd', 'stm_oorz_code', 'stm_prioriteit']
        df = df.reset_index(drop=True)
        df = df[(df['stm_hers_tijd'] >= 4) & (df['stm_hers_tijd'] <= 480)]
        df = df.reset_index(drop=True)
        return df
    
    def rfr_train_model (df, dummies_columns):
        X_train = pd.get_dummies(data=df[dummies_columns]) # ['stm_oorz_code', 'stm_prioriteit']
        y_train = df['stm_hers_tijd']
        regr = RandomForestRegressor(max_depth=12, random_state=0)
        regr.fit(X_train, y_train)
        return X_train, y_train, regr
    
    df_RFR = prep_data(df[['#stm_sap_meldnr', 'stm_hers_tijd', 'stm_oorz_groep','stm_prioriteit', 'stm_oorz_code']], ['stm_hers_tijd', 'stm_oorz_code', 'stm_prioriteit'])
    X_train, y_train, regr = rfr_train_model (df_RFR, dummies_columns)
    return X_train, y_train, regr
    
def RandomForestTester (oorzaakcode, prioriteit, geschatte_tijd, X_train, y_train, regr):
    def dataframe_creator(oorzaakcode, prioriteit, geschatte_tijd):
        X_test = pd.DataFrame({'stm_oorz_code': [oorzaakcode], 'stm_prioriteit': [prioriteit]})
        y_test = pd.DataFrame({'stm_hers_tijd': [geschatte_tijd]})
        return X_test, y_test
    
    def predictor(regr, X_train, y_train, X_test, y_test):
        y_pred = regr.predict(X_test)
        meansquare = np.sqrt(metrics.mean_squared_error(y_test,y_pred))
#         if geschatte_tijd != 0:
#             output = "Bij oorzaakcode {}, prioriteit {} met werkelijke tijd {} gokt het: {}, dus Hij zit er {} naast".format(oorzaakcode, prioriteit, geschatte_tijd, round(y_pred[0]), round(meansquare))
#         else:
#             output = "Bij oorzaakcode {}, prioriteit {} met onbekende werkelijke tijd gokt het: {}".format(oorzaakcode, prioriteit, round(y_pred[0]))
        output = round(y_pred[0])
        return output
    X_test, y_test = dataframe_creator(oorzaakcode, prioriteit, geschatte_tijd)
    output = predictor(regr, X_train, y_train, X_test, y_test)
    print(output)
    return output

def onetimerun():
    df = pd.read_csv('sap_storing_data_hu_project.csv', low_memory = False)
    df = df.drop_duplicates(subset=['#stm_sap_meldnr'])
    df = df.reset_index(drop=True)

    df['stm_aanntpl_tijd']= pd.to_datetime(df['stm_aanntpl_tijd'])
    df['stm_fh_ddt']= pd.to_datetime(df['stm_fh_ddt'])

    # Calculate the true reparation time
    df['stm_hers_tijd'] = (df['stm_fh_ddt'].dt.hour * 60 + df['stm_fh_ddt'].dt.minute) - (df['stm_aanntpl_tijd'].dt.hour * 60 + df['stm_aanntpl_tijd'].dt.minute)

    df = df.dropna(subset=['stm_hers_tijd']) 
    df = df.reset_index(drop=True)
    return df

In [3]:
#only 1 time run
df = onetimerun()

In [4]:
#only 1 time run
X_train, y_train, regr = Model_trainer_RandomForest(df, ['stm_oorz_code', 'stm_prioriteit'])

In [5]:
oorzaakcode = 218
prioriteit = 2
geschatte_tijd = np.nan

In [57]:

try:
    print(RandomForestTester (oorzaakcode, prioriteit, geschatte_tijd, X_train, y_train, regr))
except ValueError:
    print(RandomForestTester (oorzaakcode, prioriteit, 0, X_train, y_train, regr))

48
48


### Applicatie

In [85]:
import pandas as pd
import numpy as np
import tkinter as tk
from tkinter import *

In [86]:
oorzaakCode2 = ""
prioriteit2 = ""
calculatedTime = ""

In [87]:
def application():
    root = tk.Tk()

    root.title("ProRail applicatie HBO-AI 2020 groep 3")
    proRailLogoColour = '#CD1E44'
    backgroundColour = '#FFFFFF'

    root.configure(background=backgroundColour)

    def activateSendButton():
        global oorzaakCode2
        global prioriteit2
        sendButton.config(text="Bereken Hersteltijd voor {}, {}".format(oorzaakCode2, prioriteit2))
        if len(oorzaakCode2) != 0:
            if len(prioriteit2) != 0:
                sendButton.config(state=NORMAL)

    def oorzOnClick():
        global oorzaakCode2
        oorzaakCode2 = oorzEntry.get()
        oorzLabel.config(text="Oorzaakcode: {}".format(oorzaakCode2))
        activateSendButton()
        print(calculatedTime)
    
    

    def prioOnClick():
        global prioriteit2
        prioriteit2 = prioEntry.get()
        prioLabel.config(text="Prioriteit: {}".format(prioriteit2))
        activateSendButton()

    def sendInfo():
        sendButton.config(state=DISABLED)
        # Hier output aftappen !!!
        global oorzaakCode2
        global prioriteit2
        try:
            calculatedTime = "{} Minuten".format(RandomForestTester (oorzaakCode2, prioriteit2, geschatte_tijd, X_train, y_train, regr))
        except ValueError:
            calculatedTime = "{} Minuten".format(RandomForestTester (oorzaakCode2, prioriteit2, 0, X_train, y_train, regr))
        herstel2Label.config(text='{}'.format(calculatedTime))
    
        oorzaakCode2 = ""
        prioriteit2 = ""
        oorzLabel.config(text="Oorzaakcode: {}".format(oorzaakCode2))
        prioLabel.config(text="Prioriteit: {}".format(prioriteit2))
        sendButton.config(text="Bereken Hersteltijd")

    BlankFrame = Frame(master=root, bg=proRailLogoColour, width=800, height=10, pady=30)
    BlankFrame.pack(fill=BOTH)

    bgList = Frame(master=root, bg=backgroundColour, width=800)
    bgList.pack(fill=BOTH)

    meldnrFrame = Frame(master=bgList, bg=proRailLogoColour, width=800, height=0, pady=10)
    meldnrFrame.pack(fill=BOTH, side=TOP)

    meldnrLabel = Label(master=meldnrFrame, text="Meldingnummer: {}".format("meldnr"), 
                        foreground="black", background=backgroundColour, font='Bold, 20', anchor=W)
    meldnrLabel.pack(fill=BOTH, padx=50)



    oorzFrame = Frame(master=bgList, bg=proRailLogoColour, width=800, height=0, pady=10)
    oorzFrame.pack(side=TOP)

    oorzLabel = Label(master=oorzFrame, text="Oorzaakcode: {}".format(oorzaakCode2), 
                        foreground="black", background=proRailLogoColour, font='Bold, 18', anchor=W)
    oorzLabel.pack(padx=30, anchor=NW, side=TOP)

    invoerFrame = Frame(master=oorzFrame, bg=proRailLogoColour, width=600, height=0)
    invoerFrame.pack(fill=BOTH, side=TOP)

    oorzEntry = Entry(master=invoerFrame, width=20, justify=LEFT, relief=RAISED, font='Bold, 16')
    oorzEntry.pack(fill=BOTH, side=LEFT, pady=10, padx=30)

    oorzButton = Button(master=invoerFrame, command=oorzOnClick, text="Accepteer", height=3, width=10)
    oorzButton.pack(fill=BOTH, side=RIGHT, pady=10, padx=10)



    prioFrame = Frame(master=bgList, bg=proRailLogoColour, width=800, height=0, pady=10)
    prioFrame.pack(side=TOP)

    prioLabel = Label(master=prioFrame, text="Prioriteit: {}".format(prioriteit2),
                        foreground="black", background=proRailLogoColour, font='Bold, 18', anchor=W)
    prioLabel.pack(padx=30, anchor=NW, side=TOP)

    invoerFrame2 = Frame(master=prioFrame, bg=proRailLogoColour, width=600, height=0)
    invoerFrame2.pack(fill=BOTH, side=TOP)

    prioEntry = Entry(master=invoerFrame2, width=20, justify=LEFT, relief=RAISED, font='Bold, 16')
    prioEntry.pack(fill=BOTH, side=LEFT, pady=10, padx=30)

    prioButton = Button(master=invoerFrame2, command=prioOnClick, text="Accepteer", height=3, width=10)
    prioButton.pack(fill=BOTH, side=RIGHT, pady=10, padx=10)



    sendFrame = Frame(master=bgList, bg=backgroundColour, width=800, height=0, pady=10)
    sendFrame.pack(side=TOP)

    sendButton = Button(master=sendFrame, command=sendInfo, text="Bereken Hersteltijd", height=3, state=DISABLED)
    sendButton.pack(fill=BOTH, pady=10, padx=20)



    # Reparatietijd/hersteltijd en betrouwbaarheid

    herstelFrame = Frame(master=bgList, bg=proRailLogoColour, width=800, height=0, pady=10)
    herstelFrame.pack(side=TOP)

    herstelLabel = Label(master=herstelFrame, text='Berekende hersteltijd:', font='Bold, 16', anchor=W, bg=proRailLogoColour)
    herstelLabel.pack(padx=30, side=TOP)

    herstel2Label = Label(master=herstelFrame, text='{}'.format(calculatedTime), font='Bold, 26', anchor=W)
    herstel2Label.pack(padx=30, pady=40, side=TOP)

    betrLabel = Label(master=herstelFrame, text='Betrouwbaarheid:', font='Bold, 16', anchor=W, bg=proRailLogoColour)
    betrLabel.pack(padx=30, side=TOP)

    betr2Label = Label(master=herstelFrame, text='{}'.format("calculatedBetr"), font='Bold, 26', anchor=W)
    betr2Label.pack(padx=30, pady=40, side=TOP)



    root.mainloop()

In [88]:
application()